In [1]:
import json

In [7]:
!pip install pypdf2

In [2]:
from PyPDF2 import PdfReader

In [59]:
def text_extractor(path: str):
    reader = PdfReader(path)
    # page = reader.pages[0]
    return reader.pages
    # return page.extract_text()

In [120]:
def process_data_dictionary(pages):
    data = []
    for page in pages:
        txt = page.extract_text()
        ntext = txt.split("\n") # Split text by jumplines
        for i, t in enumerate(ntext):
            if "Valo r" in t:
                text = ntext[i+2].split()
                if len(text) > 1 and "Etiqueta" in text:
                    nquestion = text[1] # Number of question
                    question = " ".join(text[2:]) # Text question
                    qtype = ntext[i+3] # Question type
                    if "Tipo" not in qtype: # The question continue
                        question += " " + "".join(ntext[i+3])
                        qtype = ntext[i+4].split()[1] if "Tipo" in ntext[i+4] else None # Updating Question type
                        rtype = ntext[i+6].split()[1] if "Medición" in ntext[i+6] else None
                        n_value = i+7 # Next Value
                    else:
                        qtype = qtype.split()[1] # Removing 'Tipo'
                        rtype = ntext[i+5].split()[1] if "Medición" in ntext[i+5] else None
                        n_value = i+6 # Next Value
                    # Get answers options 
                    options = {}
                    try:
                        while "Valo" not in ntext[n_value] and rtype:
                            opt = ntext[n_value].split()
                            options[opt[0].strip()] = " ".join(opt[1:])
                            n_value += 1
                    except Exception as e: # Ignore 2 questions (implies much effort to fix)
                        print(f"Exception: {e}\nNLINE: {n_value}\nText:{t}\n")
                    data.append({
                        "nq": nquestion,
                        "question": question,
                        "qtype": qtype,
                        "rtype": rtype,
                        "options": options

                    })
                    n_value = 0
        # Serializing json
    json_object = json.dumps(data, indent=4, ensure_ascii=False)
        
    # Writing to sample.json
    with open("data_dict_e_nal.json", "w", encoding="utf-8") as outfile:
        outfile.write(json_object)

In [121]:
path = "diccionario20de20codigos.pdf"
pages = text_extractor(path)
# print(pages[0].extract_text())

In [122]:
print("Generating json..")
process_data_dictionary(pages)

Generating json..
list index out of range 
NLINE: 66
Valo r
list index out of range 
NLINE: 67
Valo r
